## generate previous frames with pCO2 and fit it against ConvLSTM

### with 1 member + no socat mask best result:
Full RMSE score:
10.710741871844716
SOCAT RMSE score:
11.744262459251226

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import os

sys.path.insert(0, '../../src')

from utils import df_to_xarray,read_xarray, custom_rmse

sys.path.insert(0, '../../src/preprocess')
from data_preprocess import preprocess_image_reduced,preprocess_images_nfp, inverse_scale_frame
from data_preprocess import preprocess_images, inverse_scale_image, preprocess_image_reduced

/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
2022-08-19 00:15:48.118606: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
print(tf. __version__) 

2.4.0


In [4]:
print(np. __version__) 

1.20.0


### Previous Frame generation

In [2]:
dist_map = pd.read_csv("../../src/dist_map.csv",header=None).to_numpy()
dist_map = np.roll(np.fliplr(dist_map),180)
dist_map = np.repeat(dist_map[np.newaxis, :, : ], 421, axis=0)


dir_name = "../../data/data1"
val_dir_name = "../../data/data2"

chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(dir_name)


/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
ecCodes library not found using ['eccodes', 'libeccodes.so', 'libeccodes']
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [3]:
print(tf.config.list_physical_devices('GPU'))
gpu_available = tf.test.is_gpu_available()
gpu_available

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-08-19 00:16:27.605398: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-19 00:16:27.634127: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-19 00:16:27.849464: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-08-19 00:16:27.849775: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-08-19 00:16:30.003411: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-08-19 00:16:30.003558: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-0

True

Adding visible gpu devices: 0
2022-08-19 00:16:32.734105: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-08-19 00:16:38.678769: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1261] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-08-19 00:16:38.678800: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1267]      0 
2022-08-19 00:16:38.678810: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1280] 0:   N 
2022-08-19 00:16:38.701319: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/device:GPU:0 with 42529 MB memory) -> physical GPU (device: 0, name: Quadro RTX 8000, pci bus id: 0000:3b:00.0, compute capability: 7.5)


In [4]:
# substituting mld with dist_map

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y1 = preprocess_image_reduced(pco2.pCO2.data)
dist_map = preprocess_image_reduced(dist_map)
    

X1 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images))
X1 = X1.reshape((421,180,360,5),order='F')

INPUT_SHAPE=X1[0].shape
INPUT_SHAPE

(180, 360, 5)

In [5]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(val_dir_name, num = "035")

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y2 = preprocess_image_reduced(pco2.pCO2.data)
dist_map = preprocess_image_reduced(dist_map)
    

X2 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images))
X2 = X2.reshape((421,180,360,5),order='F')

In [6]:
import sklearn
from sklearn.model_selection import train_test_split

X=np.concatenate((X1,X2), axis = 0)
y=np.concatenate((y1,y2),axis = 0)


train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3)

In [18]:

from functools import partial
train_X.shape

(502, 3, 180, 360, 6)

In [11]:
OUTPUT_SHAPE = y[0].shape

In [15]:
tf.keras.backend.clear_session()

from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=5,activation='elu',
                        padding="SAME")

base_model = keras.models.Sequential([
    DefaultConv2D(filters=32, input_shape=INPUT_SHAPE),
    DefaultConv2D(filters=32),
    keras.layers.MaxPooling2D(pool_size=3),
    keras.layers.Dropout(0.2),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    keras.layers.MaxPooling2D(pool_size=3),
    keras.layers.Dropout(0.2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.UpSampling2D(size=3),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    keras.layers.Dropout(0.2),
    keras.layers.UpSampling2D(size=3),
    DefaultConv2D(filters=32),
    DefaultConv2D(filters=2),
    DefaultConv2D(filters=1,kernel_size=1),
    keras.layers.Reshape(OUTPUT_SHAPE)
])

myLearnRate=0.0005
custom_opt = tf.keras.optimizers.Adam(learning_rate=myLearnRate)
base_model.compile(loss=custom_rmse, optimizer=custom_opt)

In [16]:
import warnings
warnings.filterwarnings(action='once')

model_path="../../models/base_model/u_net_dist.h5"
early_stopings = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

history = base_model.fit(train_X,train_y, epochs=500, 
                         validation_data=(val_X,val_y),
                         workers=-1,batch_size=32,
                         callbacks=callbacks)



Epoch 1/500
19/19 [==============================] - ETA: 0s - loss: 62.2675

2022-08-18 23:32:58.503067: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2022-08-18 23:32:58.798639: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2022-08-18 23:32:59.096097: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2022-08-18 23:32:59.406448: W tensorflow/stream_executor/gpu/

19/19 [==============================] - 70s 4s/step - loss: 61.4650 - val_loss: 19.9586


2022-08-18 23:33:59.858285: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'



Epoch 2/500
19/19 [==============================] - 4s 193ms/step - loss: 20.4052 - val_loss: 18.2267
Epoch 3/500
19/19 [==============================] - 3s 172ms/step - loss: 16.2824 - val_loss: 16.1627
Epoch 4/500
19/19 [==============================] - 3s 170ms/step - loss: 14.3507 - val_loss: 14.9028
Epoch 5/500
19/19 [==============================] - 3s 173ms/step - loss: 12.5675 - val_loss: 11.5477
Epoch 6/500
19/19 [==============================] - 3s 171ms/step - loss: 11.1474 - val_loss: 10.6636
Epoch 7/500
19/19 [==============================] - 3s 171ms/step - loss: 10.1690 - val_loss: 9.7221
Epoch 8/500
19/19 [==============================] - 3s 170ms/step - loss: 9.1829 - val_loss: 7.9844
Epoch 9/500
19/19 [==============================] - 3s 170ms/step - loss: 8.8689 - val_loss: 9.2272
Epoch 10/500
19/19 [==============================] - 3s 170ms/step - loss: 8.1302 - val_loss: 8.7100
Epoch 11/500
19/19 [==============================] - 3s 170ms/step - loss: 7.9

In [8]:
cnn_model = tf.keras.models.load_model('../../models/base_model/u_net_dist.h5', custom_objects={'custom_rmse':custom_rmse})
predicted_image= cnn_model.predict(X1,verbose=1)
predicted_image[y1==0]=0.0

2022-08-19 00:17:08.353109: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-19 00:17:08.355106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-08-19 00:17:08.355175: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-08-19 00:17:08.355193: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-08-19 00:17:08.355202: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-08-19 00:17:08.355210: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022

14/14 [==============================] - 94s 109ms/step


In [9]:
pred_val= cnn_model.predict(X2,verbose=1)
pred_val[y2==0]=0.0

14/14 [==============================] - 1s 71ms/step


In [10]:
print(custom_rmse(predicted_image,y1))
print(custom_rmse(predicted_image,y1))

tf.Tensor(85.21258296542308, shape=(), dtype=float64)
tf.Tensor(85.21258296542308, shape=(), dtype=float64)


In [11]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2t2 = read_xarray(val_dir_name,num="035")

y_true,y_pred = inverse_scale_image(pred_val,pco2t2.pCO2.data)


y_pred_socat = np.copy(y_pred)
y_true_socat = np.nan_to_num(pco2t2.pCO2_socat.data)
y_pred_socat[y_true_socat==0]=0.0


print("val Full RMSE score:")
a=custom_rmse(y_pred,y_true)
print(a.numpy())

print("val SOCAT RMSE score:")
b=custom_rmse(y_pred_socat,y_true_socat)
print(b.numpy())

val Full RMSE score:
8.378366875091343
val SOCAT RMSE score:
8.80500100918264


In [12]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2t2 = read_xarray(dir_name)

y_true,y_pred = inverse_scale_image(predicted_image,pco2t2.pCO2.data)

y_pred_socat = np.copy(y_pred)
y_true_socat = np.nan_to_num(pco2t2.pCO2_socat.data)
y_pred_socat[y_true_socat==0]=0.0


print("Full RMSE score:")
a=custom_rmse(y_pred,y_true)
print(a.numpy())

print("SOCAT RMSE score:")
b=custom_rmse(y_pred_socat,y_true_socat)
print(b.numpy())

Full RMSE score:
7.988057944417932
SOCAT RMSE score:
8.42041374977283


### Using the prediction as input in ConvLSTM

In [13]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(dir_name)

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y1 = preprocess_image_reduced(pco2.pCO2_socat.data)
dist_map = preprocess_image_reduced(dist_map)
    

Xc1 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images,predicted_image))
Xc1 = Xc1.reshape((421,180,360,6),order='F')

In [14]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(val_dir_name,num="035")

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y2 = preprocess_image_reduced(pco2.pCO2_socat.data)
dist_map = preprocess_image_reduced(dist_map)
    

Xc2 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images,pred_val))
Xc2 = Xc2.reshape((421,180,360,6),order='F')

In [15]:
X=np.concatenate((Xc1, Xc2), axis = 0)
y=np.concatenate((y1,y2),axis = 0)

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.4)

In [16]:
import numpy as np

train_X_index=np.lib.stride_tricks.sliding_window_view(range(train_X.shape[0]),3) 
val_X_index=np.lib.stride_tricks.sliding_window_view(range(val_X.shape[0]),3) 

train_y=np.expand_dims(train_y[train_X_index][1:],axis=4)
train_X=train_X[train_X_index][:-1]

val_y=np.expand_dims(val_y[val_X_index][1:],axis=4)
val_X=val_X[val_X_index][:-1]

print("train set shape")
print(train_X.shape, train_y.shape)

print("val set shape")
print(val_X.shape, val_y.shape)

INPUT_SHAPE = train_X[0].shape


train set shape
(502, 3, 180, 360, 6) (502, 3, 180, 360, 1)
val set shape
(334, 3, 180, 360, 6) (334, 3, 180, 360, 1)


In [30]:
tf.keras.backend.clear_session()

DefaultConvLSTM2D = partial(keras.layers.ConvLSTM2D,
                        filters=32, kernel_size=(5, 5),
                        padding="same",return_sequences=True,
                        activation="elu",)



model = keras.models.Sequential([
    DefaultConvLSTM2D(input_shape=INPUT_SHAPE),
    keras.layers.BatchNormalization(),
    DefaultConvLSTM2D(kernel_size=(3,3)),
    keras.layers.BatchNormalization(),
    DefaultConvLSTM2D(kernel_size=(1,1)),
    keras.layers.Conv3D(filters = 1, kernel_size=(3,3,3),activation="elu", padding="same")
    
])


model.compile(
    loss=custom_rmse, optimizer=keras.optimizers.Adam(learning_rate=0.007),
)
# tried 0.005, 0.001, 0.0008

In [ ]:
model_path="../../models/ConvLSTM_with_pco2.h5"

early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

epochs = 50
batch_size = 8

# Fit the model to the training data.
hist = model.fit(
    train_X,
    train_y,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_X,val_y),
    callbacks=callbacks,
)

In [ ]:
X_index=np.lib.stride_tricks.sliding_window_view(range(421),3) 
Xc1 = Xc1[X_index][:-1]
y1=np.expand_dims(y1[X_index][1:],axis=4)

best_model = tf.keras.models.load_model('../../models/ConvLSTM_with_pco2.h5', custom_objects={'custom_rmse':custom_rmse})
pred1=best_model.predict(Xc1,verbose=1)
pred1[y1==0]=0.0


chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2t2 = read_xarray(dir_name)

y_true,y_pred = inverse_scale_frame(pred1,pco2t2.pCO2.data)

y_pred_socat = np.copy(y_pred)
y_true_socat=np.expand_dims(pco2t2.pCO2_socat.data[X_index][1:],axis=4)
y_true_socat = np.nan_to_num(y_true_socat)
y_pred_socat[y_true_socat==0]=0.0

In [ ]:
print("Full RMSE score:")
a=custom_rmse(y_pred[:,:2],y_true[:,:2])
print(a.numpy())

print("SOCAT RMSE score:")
b=custom_rmse(y_pred_socat[:,:2],y_true_socat[:,:2])
print(b.numpy())

In [ ]:
import imageio
import matplotlib.colors as mcolors


d = y_pred-y_true


norm = mcolors.TwoSlopeNorm(vmin=d.min(), vmax = d.max(), vcenter=0)


filenames = []

for i in range(418):
    # plot the line chart
    figure, axis = plt.subplots(2, 2,figsize=(12, 6))

    img=axis[0][0].imshow(np.flipud(y_pred[i][1]),cmap="coolwarm", interpolation="nearest")
    axis[0][0].set_title("prediction")
    plt.colorbar(img,ax=axis)

    img1=axis[0][1].imshow(np.flipud(y_true[i][1]),cmap="coolwarm", interpolation="nearest")
    axis[0][1].set_title("true")

    diff=np.flipud(np.squeeze(y_pred[i][1]-y_true[i][1]))
    img2=axis[1][0].imshow(diff,cmap="RdBu", interpolation="nearest",norm=norm)
    axis[1][0].set_title("residual")
    plt.colorbar(img2,ax=axis)
    
    img2=axis[1][1].imshow(np.flipud(Xc1[i][1][:,:,5]),cmap="coolwarm", interpolation="nearest")
    axis[1][1].set_title("input: previous predicted pco2")
    # create file name and append it to a list
    filename = f'{i}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()


with imageio.get_writer('../../assets/cnn-lstm-new.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)